In [ ]:
#we need to import the needed libraries
from sklearn.model_selection import train_test_split
import numpy as np
from PIL import Image
import glob 
import csv
from keras.layers import Dense,Dropout
from keras.models import Sequential
from keras.layers import Activation,Conv2D,MaxPooling2D,Flatten,BatchNormalization, Input
from keras import utils
from keras import optimizers
import pandas as pd
import keras
import MacOSFile as msf
from keras.preprocessing.image import ImageDataGenerator


In [ ]:
# the goal of this is to basically get the data into the form that I have in the other situation.
x = msf.pickle_load("whales.pkl")
df = pd.read_csv('input/train.csv')
y = df['Id'].tolist()

In [ ]:
count = 810
i = 0
while i < len(x):
    if y[i] == 'new_whale' and count > 600:
        del y[i]
        del x[i]
        count -= 1
    else:
        i += 1

In [ ]:
#convert the labels in y to numbers to do categorical classification on. 
id_to_num = {}
num_to_id = {}
i = 0
for elem in np.unique(y):
    id_to_num[elem] = i
    num_to_id[i] = elem
    i += 1
#check that the dicitonary has 4251 entries

#I'm just turning each code into its own apporpriate integer.
def convert_to_num(lis):
    l = []
    for i in range(len(lis)):
        id = lis[i]
        l.append(id_to_num[id])
    return l

def convert_to_id(lis):
    l = []
    for i in range(len(lis)):
        num = lis[i]
        print(num, num_to_id[num])
        l.append(num_to_id[num])
    return l
#lets test this to make sure it works, we can extract
#the location of where the
y = convert_to_num(y)

#I am now trying to achieve this through keras with their system
y = utils.to_categorical(y, num_classes=4251)

train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)


In [ ]:
x = np.asarray(x)

In [ ]:
# for i in range(len(x)):
#     x[i] = np.asarray(x[i])
print(x.shape, type(x))
print(y.shape, type(y))
print(x[0].shape, type(x[0]))


In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x,y,random_state=20)

In [ ]:
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

In [ ]:
train_datagen.fit(x_train)
test_input = msf.pickle_load("whales_test.pkl")
files = glob.glob('input/test/*.jpg')
test_input = np.asarray(test_input)
test_input.shape

In [ ]:
#I think the data might be in the correct format now. 
#now lets create the model
adam = optimizers.Adam(lr=0.0001)
model = Sequential()
model.add(Conv2D(32,(3,3),padding='same',
                 data_format = "channels_last",input_shape=(64,64,3)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
model.add(Conv2D(32,(3,3),padding='same',
                 data_format = "channels_last",input_shape=(64,64,3)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
model.add(Conv2D(32,(3,3),padding='same',
                 data_format = "channels_last",input_shape=(64,64,3)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

# model.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
# model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
# model.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
# model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
# model.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
# model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
# model.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
# model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
model.add(Dropout(rate=.10))
model.add(Flatten())
model.add(Dense(100))
model.add(Dense(4251, activation='softmax'))
model.compile(optimizer = adam, loss = 'categorical_crossentropy', metrics = ['accuracy'])
model.summary()

In [ ]:
# fits the model on batches with real-time data augmentation:
model.fit_generator(train_datagen.flow(x, y, batch_size=500),
                    steps_per_epoch=len(x)/500, epochs=3)
# #now lets try and fit the model to the data, we have the same error
# model.fit(x_train, y_train,
#           batch_size=512,
#           epochs=3,
#           verbose=1,
#           validation_data=(x_test, y_test))
#score = model.evaluate(x_test, y_test, verbose=0)
#print(score)


In [ ]:
results = model.predict(test_input)

In [ ]:
import warnings
from os.path import split

with open("sample_submission.csv","w") as f:
    with warnings.catch_warnings():
        f.write("Image,Id\n")
        warnings.filterwarnings("ignore",category=DeprecationWarning)
        for file,img in zip(files,test_input):
            file_name = file[len("input/test/"):]
            results = model.predict_proba(img.reshape(1,64,64,3))
            predicted_args = np.argsort(results)[0][::-1][:5]
            print(predicted_args)
            predicted_args = convert_to_id( predicted_args)
            predicted_args = " ".join( predicted_args)
            f.write("%s,%s\n" %(file_name, predicted_args))